#  프로젝트 - Movielens 영화 추천 

```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip

```

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 시청횟수로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.


# 1) 데이터 준비와 전처리

```
ratings            : 유저id,무비id,평점,timestamp
movies             : 무비id,영화명,장르
orginal_data_size  : 평점이 1~5인 영화 갯수(1000209)
filtered_data_size : 평점이 3이상인 영화 갯수(836478) 
```

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# 2) 분석해 봅시다.

- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

```
ratings            : ['user_id', 'movie_id', 'count', 'timestamp']
movies             : ['movie_id', 'title', 'genre']
movies_data        : ['user_id', 'movie_id', 'count', 'timestamp', 'title', 'genre']
orginal_data_size  : 평점이 1~5 횟수  (1000209)
filtered_data_size : 평점이 3이상인 횟수(836478) 
```

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [7]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

#### 두 DataFrame(ratings과 movies)를 기준 컬럼(movie_id) 기준으로 Join
https://programmerpsy.tistory.com/17

In [8]:
movies_data = pd.merge(left = ratings , right = movies, how = "inner", on = "movie_id")
movies_data.head(10)

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,One Flew Over the Cuckoo's Nest (1975),Drama
7,24,1193,5,978136709,One Flew Over the Cuckoo's Nest (1975),Drama
8,28,1193,3,978125194,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,One Flew Over the Cuckoo's Nest (1975),Drama


#### 결측치 확인

In [9]:
movies_data.isnull().sum()

user_id      0
movie_id     0
count        0
timestamp    0
title        0
genre        0
dtype: int64

### 가장 인기 있는 영화 30개(인기순)

In [10]:
# 평점 3,4,5 모두 인기있는 경우라고 정한다.
# 3,4,5점을 한번 받으면 1 count로 정하고, 각 영화에 대한 총 count수를 로 정렬한다. 
movies_count = movies_data.groupby('title')['count'].count()

movies_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

판다스 함수    
https://dandyrilla.github.io/2017-08-12/pandas-10min/#2-%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%99%95%EC%9D%B8%ED%95%98%EA%B8%B0-viewing-data




```
moives_data   :'user_id','movie_id','count','timestamp','title','genre'
my_favorite   : 5개의 영화 title 리스트
dir_favorite  : {'title' :'movie_id'}
my_moivelist  : 

```

### title 입력시 -> movie_id 반환 함수

In [11]:
# [title] input -> [movie_id] output
def title_in_id_out(title):
    a=movies_data[movies_data['title'].isin([title])] # title칼럼중 인자로 들어온 title과 같은 행렬 추출
    mv_id=a.iloc[0,1] #추출된 행렬중 0번째 행의 1번째칼럼[movie_id] 값 가져온다.
    return mv_id
#title가 'Jumanji (1995)'인 movie_id 출력
title_in_id_out('Jumanji (1995)')

2

### movie_id 입력시 -> title  반환 함수¶

In [12]:
# [movie_id] output -> [title] input
def id_in_title_out(movie_id):
    a=movies_data[movies_data['movie_id'].isin([movie_id])] # movie_id칼럼중 인자에 들어온 영화id와 같은 행렬 추출
    title=a.iloc[0,4] #추출된 행렬중 0번째 행의 4번째칼럼[title] 값 가져온다.
    return title

#moive_id가 23인 title값 출력
id_in_title_out(23)

'Assassins (1995)'

### 선호하는 영화 리스트 ,딕셔너리 만들기( title과 movie_id )

In [13]:
# 영화 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
#선호 영화 리스트 생성
my_favorite = ['Toy Story (1995)' , 'Terminator, The (1984)' ,'Matrix, The (1999)' ,'Toy Story 2 (1999)' ,'Jumanji (1995)']
my_movie_id = []


#선호 영화 딕셔너리 생성
dir_favorite ={} #{'title' :'movie_id'} 딕셔너리
 
for title in my_favorite:
    mv_id = title_in_id_out(title)
    my_movie_id.append(mv_id) 
    print('영화명:',title,'\t\t 무비id:', mv_id)
    
    dir_favorite[title]=int(mv_id)
    print('영화명:',title,'\t\t 무비id:', dir_favorite[title])
    
       

#print(movies_data.columns)
#movies_data.sort_values(by='movie_id')
my_movie_id

영화명: Toy Story (1995) 		 무비id: 1
영화명: Toy Story (1995) 		 무비id: 1
영화명: Terminator, The (1984) 		 무비id: 1240
영화명: Terminator, The (1984) 		 무비id: 1240
영화명: Matrix, The (1999) 		 무비id: 2571
영화명: Matrix, The (1999) 		 무비id: 2571
영화명: Toy Story 2 (1999) 		 무비id: 3114
영화명: Toy Story 2 (1999) 		 무비id: 3114
영화명: Jumanji (1995) 		 무비id: 2
영화명: Jumanji (1995) 		 무비id: 2


[1, 1240, 2571, 3114, 2]

### user_id 순으로 정렬하기

In [14]:
# user_id 마지막은 6040
# user_id은 1부터 시작해서 6040개로 끝난다면 유니크한 값은 6040개가 되어야되는데 6039개다 ,그럼 중간에 한개의 값이 없다고 추측된다.
print(ratings['user_id'].nunique()) 
movies_data.sort_values(by='user_id',ascending=True, inplace=False)

6039


,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
31113,1,2294,4,978824291,Antz (1998),Animation|Children's
31674,1,3186,4,978300019,"Girl, Interrupted (1999)",Drama
32044,1,1566,4,978824330,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
32415,1,588,4,978824268,Aladdin (1992),Animation|Children's|Comedy|Musical
...,...,...,...,...,...,...
657728,6040,334,4,957717503,Vanya on 42nd Street (1994),Drama
393446,6040,1294,4,957716949,M*A*S*H (1970),Comedy|War
253075,6040,994,3,960972693,Big Night (1996),Drama
127665,6040,2396,3,956704475,Shakespeare in Love (1998),Comedy|Romance


### 새로운 사용자 정보 추가
위에서 생성한 선호 영화5개를 좋아하는 syj계정을 ratings의 DataFrame에 추가한다.

In [15]:

 
#my_moivelist = pd.DataFrame({'user_id': [0]*5, 'movie_id': my_movie_id, 'count':[5]*5})
my_moivelist = pd.DataFrame({'user_id': ['syj']*5, 'movie_id': my_movie_id, 'count':[5]*5})

# 사용자는 6039이였기때문에  나에대한 유저번호는 6040으로 설정한다.
if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_moivelist)              

ratings.tail(10)

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,syj,1,5,NaN
1,syj,1240,5,NaN
2,syj,2571,5,NaN
3,syj,3114,5,NaN
4,syj,2,5,NaN


### 두 DataFrame(새로운 데이터를 갖은ratings와 movies)를 기준 컬럼(movie_id) 기준으로 Join

836478->836483 기존 ratings에서 5개의 행이 추가되었다.

In [16]:
print('기본행 개수',len(movies_data))
movies_data = pd.merge(left = ratings , right = movies, how = "inner", on = "movie_id")
movies_data.head(10)
print('추가후 행 개수',len(movies_data))
movies_data.tail(5)

기본행 개수 836478
추가후 행 개수 836483


,user_id,movie_id,count,timestamp,title,genre
836478,5851,3607,5,9.577566e+08,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,9.583469e+08,Slaughterhouse (1987),Horror
836480,5854,690,3,9.577443e+08,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,9.572734e+08,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,5948,1360,5,1.016564e+09,Identification of a Woman (Identificazione di ...,Drama


### movies_data중 활용할 칼럼만 movies_tmp(DataFrame)으로 뽑는다.

In [17]:
movies_tmp= movies_data.loc[:,['user_id','movie_id','count']]
movies_tmp

,user_id,movie_id,count
0,1,1193,5
1,2,1193,5
2,12,1193,4
3,15,1193,4
4,17,1193,5
...,...,...,...
836478,5851,3607,5
836479,5854,3026,4
836480,5854,690,3
836481,5938,2909,4


# indexing
- 데이터를 식별하기 위한 데이터 인덱싱 작업
- 우린 user_id와 movie_id 각각에 번호를 붙인다.
- user_id 경우 syj이라는 문자열이 존재한다. moive_id경우 모두 숫자이다.그러므로 user_id만 재 변경해주자.

In [18]:
# 고유한 유저, movie_id를 찾아내는 코드
user_unique = movies_tmp['user_id'].unique()
movie_unique = movies_tmp['movie_id'].unique()

# 유저id, 영화id indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

### user_id의 인덱싱정보

In [19]:
num=0
for key, value in user_to_idx.items():
    print('user_id:{}    ,user_idx:{}'.format(key, value))
    num+=1
    if num>5:
        break
print('user_to_idx의 사이즈:', len(user_to_idx))       

user_id:1    ,user_idx:0
user_id:2    ,user_idx:1
user_id:12    ,user_idx:2
user_id:15    ,user_idx:3
user_id:17    ,user_idx:4
user_id:18    ,user_idx:5
user_to_idx의 사이즈: 6040


### moive_id의 인덱싱정보

In [20]:
num=0
for key, value in movie_to_idx.items():
    print('movie_id:{}    ,movie_idx:{}'.format(key, value))
    num+=1
    if num>5:
        break
print('user_to_idx의 사이즈:', len(movie_to_idx))       

movie_id:1193    ,movie_idx:0
movie_id:661    ,movie_idx:1
movie_id:914    ,movie_idx:2
movie_id:3408    ,movie_idx:3
movie_id:2355    ,movie_idx:4
movie_id:1197    ,movie_idx:5
user_to_idx의 사이즈: 3628


### movies_tmp에 인덱싱(id,movie)된 데이터로 저장

In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movies_tmp['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movies_tmp):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movies_tmp['user_id'] = temp_user_data   # movies_tmp['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

#movie_id를 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movies_tmp['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movies_tmp):
    print('movie column indexing OK!!')
    movies_tmp['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

movies_tmp

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
836478,1621,3623,5
836479,3481,3624,4
836480,3481,3625,3
836481,4159,3626,4


# CSR matrix

## CSR란
-Compressed Sparse Row 의 약자입니다.    
-CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다. 
- https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix
- https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005
![CSR]( https://lovit.github.io/assets/figures/sparse_matrix_csr.png )

[8-6. CSR] csr_matrix의 indices 와 indptr 개념
https://rfriend.tistory.com/551
![ㅁ](https://t1.daumcdn.net/cfile/tistory/99C845445F69BC8A32)

# 4) CSR matrix를 직접 만들기

In [22]:
#SciPy(사이파이)는 과학 컴퓨팅과 기술 컴퓨팅에 사용되는 자유-오픈 소스 파이썬 라이브러리이다
from scipy.sparse import csr_matrix

num_user = movies_tmp['user_id'].nunique()
num_movie = movies_tmp['movie_id'].nunique()


#=> csr_matrix((data, indices, indptr), shape=(row, col))
csr_data = csr_matrix((movies_tmp['count'], (movies_tmp['user_id'], movies_tmp['movie_id'])), shape= (num_user, num_movie))
#csr_data

#  5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련하기
- Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있다.

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

``` 
 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 
 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
 3. use_gpu : GPU를 사용할 것인지 
 4. iterations : epochs와 같은 의미
``` 

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(\
        factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

```
 als_model          :Implicit AlternatingLeastSquares 모델
 csr_data           : user_id'와 'movie_id'를 가지고 scipy라이브러리가 적용된 csr_matrix
 csr_data_transpose : csr_data를 내적하기 쉽게 Transpose해준다.
```

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

#  6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 나의 선호 영화 정보 찾는법
  1.사용자id, 선호 영화제목로 직접 입력한다.(노드 진행방식)/이 방식으로 진행하겠다.   
  2.밑과 같이 접근한다.

In [27]:
# syj idx값 찾기
syj_idx_num = user_to_idx['syj']
print(syj_idx_num)

#syj의 데이터정보만 가져온다.
is_moives_syj = movies_tmp['user_id'] == 5677 #56
moives_syj = movies_tmp[is_moives_syj]
moives_syj

5677


,user_id,movie_id,count
40620,5677,40,5
53173,5677,50,5
124655,5677,124,5
193101,5677,200,5
362920,5677,513,5


### Feature Matrix 확인하기 (als모델로 훈련된)
```
1)모델(als_model)이 syj유저와 가수의 벡터를 확인해보자
2)두 벡터를 곱하면 어떻값이 나오는지 살펴보겠다.

als_model.user_factors[syj]                   :  syj의 Feature Matrix       
als_model.item_factors[dir_favorite['Toy Story (1995)'] : Toy Story (1995)의 Feature Matrix

user_factors[~],item_factors[~]는 AlternatingLeastSquares의 변수인가?????
```

In [28]:
syj_idx, toystory1995_idx = user_to_idx['syj'], movie_to_idx[dir_favorite['Toy Story (1995)']]
syj_vector, toystory1995_vector = als_model.user_factors[syj_idx], als_model.item_factors[toystory1995_idx]


print('syj의 인덱스:{}  ,toystory1995의 인덱스:{}'.format(syj_idx, toystory1995_idx) )
print('슝=3')



syj의 인덱스:5677  ,toystory1995의 인덱스:40
슝=3


In [29]:
syj_vector

array([ 2.75577635e-01,  1.50722221e-01, -1.26729831e-01,  3.19408208e-01,
        6.61623955e-01,  8.00710261e-01,  4.43611145e-01, -2.03582942e-01,
        8.22215497e-01, -2.92089403e-01, -7.25150228e-01, -1.54909208e-01,
        5.89912772e-01,  3.82928222e-01, -6.31511390e-01,  5.76340139e-01,
        8.08112085e-01, -4.40448582e-01,  4.24861088e-02,  5.55863738e-01,
       -4.40339625e-01,  6.26808584e-01,  7.66437352e-01, -8.76595080e-02,
        7.64390975e-02, -2.14399830e-01, -8.37233901e-01,  1.52785349e+00,
        5.03781319e-01,  2.27519590e-02,  1.05064785e+00,  3.09169024e-01,
       -1.91409618e-01,  7.43554175e-01, -4.59949434e-01, -1.02716446e+00,
       -2.60585099e-01, -7.98849985e-02,  9.08718165e-03,  1.15952089e-01,
       -1.09015845e-01, -1.09346747e+00,  5.41121177e-02, -8.24396729e-01,
       -1.40833294e+00,  1.87456265e-01,  3.74525815e-01, -4.88101959e-01,
        8.40960860e-01,  3.76989812e-01, -8.86246979e-01, -7.77499285e-04,
        4.74207371e-01, -

In [30]:
toystory1995_vector

array([ 0.01584552,  0.00124755,  0.01059674,  0.03097196,  0.03855089,
        0.02645783,  0.01210599,  0.00439516,  0.0178262 , -0.00436286,
       -0.00720635, -0.00264205, -0.00694914,  0.028927  , -0.01144978,
        0.01617156,  0.04634731, -0.01475057,  0.02368647,  0.02121501,
       -0.01364448, -0.00399538,  0.01202685, -0.01337743, -0.01853056,
       -0.01561294, -0.01535387,  0.01987721,  0.02097918,  0.01067194,
        0.03924847,  0.01431063, -0.00453917,  0.00752472, -0.01178272,
       -0.01316751,  0.00600548, -0.01334532, -0.00352343,  0.0041913 ,
        0.02143768, -0.01785178,  0.02163541, -0.03636651, -0.01095393,
       -0.006711  , -0.00493587,  0.00292111,  0.0085131 ,  0.01307432,
       -0.04283176,  0.02007706,  0.04570417, -0.02712072,  0.01087983,
       -0.01431392,  0.00901246, -0.00993346, -0.02069543,  0.03849772,
        0.03458782,  0.00142369,  0.00527123,  0.01204033,  0.0294358 ,
       -0.01950151,  0.00483327,  0.01067477,  0.01927969,  0.00

In [31]:
# syj과 toy story 1995 내적하는 코드
np.dot(syj_vector, toystory1995_vector)

0.68336535

### 선호 이외의 영화 비교해보기 'Men in Black (1997)'

In [32]:
men_in_black_1997   = movie_to_idx[title_in_id_out('Men in Black (1997)')]
print(men_in_black_1997)              
men_in_black_1997_vector = als_model.item_factors[men_in_black_1997]

# syj과 'Men in Black (1997)' 내적하는 코드
np.dot(syj_vector, men_in_black_1997_vector)

175


0.13536981

내가 좋아하는  toy story 1995보다 더좋은 내적값이 나왔다. 즉 나와 취향이 동일한 사용자들은 'Men in Black (1997)'를  좋아한다고 추측할수 있다.

#  7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [33]:
#AlternatingLeastSquares 클래스 
#similar_items 메서드를 통하여 비슷한 영화를 찾습니다
print(my_favorite[1])

favorite_movie = 'Terminator, The (1984)'
movie_id = movie_to_idx[title_in_id_out(favorite_movie)]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie
#(영화의 idx, 유사도) Tuple 로 반환

Terminator, The (1984)


[(200, 1.0000001),
 (651, 0.7845759),
 (194, 0.6933006),
 (193, 0.6548679),
 (865, 0.64831036),
 (92, 0.61977553),
 (124, 0.5843173),
 (928, 0.5118662),
 (62, 0.5115304),
 (882, 0.5046484)]

In [34]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를  얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

for i in similar_movie:
    idx = i[0]
    moive_id = idx_to_movie[idx]
    print('idx:{},   moive_id:{} '.format(idx,moive_id))
    print(id_in_title_out(moive_id),'\n')


idx:200,   moive_id:1240 
Terminator, The (1984) 

idx:651,   moive_id:1200 
Aliens (1986) 

idx:194,   moive_id:1036 
Die Hard (1988) 

idx:193,   moive_id:1214 
Alien (1979) 

idx:865,   moive_id:3527 
Predator (1987) 

idx:92,   moive_id:589 
Terminator 2: Judgment Day (1991) 

idx:124,   moive_id:2571 
Matrix, The (1999) 

idx:928,   moive_id:2985 
Robocop (1987) 

idx:62,   moive_id:2916 
Total Recall (1990) 

idx:882,   moive_id:3703 
Mad Max 2 (a.k.a. The Road Warrior) (1981) 



#  8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

recommend메서드를 통해 유저가 좋아하는 영화를 추천받을수 있다.    
filter_already_li아ked_items =true는 유저가 이미 평가한 아이템을 제거한다.

In [55]:
user = user_to_idx['syj']
# recommend에서는 user*[???] CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=7, filter_already_liked_items=True)
movie_recommended

[(92, 0.5014723),
 (4, 0.36299428),
 (651, 0.32149833),
 (62, 0.32128203),
 (458, 0.30617693),
 (33, 0.26538277),
 (141, 0.2608834)]

### syj유저가 좋아할 만한 영화 추천

In [56]:
#movie_to_idx 를 뒤집어, index로부터 movie_id를  얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
for i in movie_recommended:
    idx = i[0]
    moive_id = idx_to_movie[idx]
    print('idx:{},   moive_id:{} '.format(idx,moive_id))
    print(id_in_title_out(moive_id),'\n')



idx:92,   moive_id:589 
Terminator 2: Judgment Day (1991) 

idx:4,   moive_id:2355 
Bug's Life, A (1998) 

idx:651,   moive_id:1200 
Aliens (1986) 

idx:62,   moive_id:2916 
Total Recall (1990) 

idx:458,   moive_id:367 
Mask, The (1994) 

idx:33,   moive_id:588 
Aladdin (1992) 

idx:141,   moive_id:457 
Fugitive, The (1993) 



aladdin1992영화를 추천받고 있다.   
explain 매소드를 사용하면 유저가 기록을 남긴 데이터 중 이 추천에 기영한 정도를 알수 있다.

In [85]:
aladdin1992   = movie_to_idx[title_in_id_out('Aladdin (1992)')]
explain = als_model.explain(user, csr_data, itemid=aladdin1992)

 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환한다.


In [86]:
[(id_in_title_out(idx_to_movie[i[0]]), i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.178152230616999),
 ('Toy Story 2 (1999)', 0.07045848351997247),
 ('Jumanji (1995)', 0.01572862127026238),
 ('Terminator, The (1984)', 0.00946572202662881),
 ('Matrix, The (1999)', -0.013866678657256572)]

```
하다가 정리한것 마지막부분의 변수는 안적음

변수
ratings            : 유저id,무비id,평점,timestamp(DataFrame)
movies             : 무비id,영화명,장르(DataFrame)
orginal_data_size  : 평점이 1~5인 영화 갯수(1000209)(int)
filtered_data_size : 평점이 3이상인 영화 갯수(836478)(int)


ratings에 있는 유니크한 영화 개수 : 3628
ratings에 있는 유니크한 사용자 수 : 6039(syj추가되면 6040)

movies_data : ratings과 movies join한다 

movies_count : 평점 3,4,5대상 title,count DataFrame

my_favorite     :  선택한 선호 영화 5개 리스트[title]
my_movie_id   : 선택한 선호 영화 5개 리스트[moive_id]
dir_favorite     : 선택한 선호 영화 5개  딕셔너리{'title' :'movie_id'} 

my_moivelist :  새로운사용자(5개의 선호영화포함) 영화정보데이터(user_id,	movie_id, count, timestamp)DataFrame
movies_tmp  : 연산될 칼럼만 저장된 데이터  (user_id, movie_id, count)Dataframe

user_unique ,num_user :  고유한 유저 수
movie_unique, num_movie : 고유한 영화 수
user_to_idx      :  인덱싱작업이 된 유저id 딕셔너리(id : idx)
movie_to_idx  : 인덱싱작업이 된 영화id 딕셔너리 (id : idx)
movies_tmp   : 인덱싱작업이된 user_id와 moive_id 저장  (user_id, movie_id, count)DataFrame
als_model         : AlternatingLeastSquares 모델

함수
title_in_id_out(title)              :title 입력시 -> movie_id 반환 함수
id_in_title_out(movie_id)  :movie_id 입력시 -> title 반환 함수

```

#  새로 학습한 내용

#### 판다스 함수    
Series.describe() -> 그룹의 기술 통계량 출력     
DataFrame.groupby함수    
data.groupby('user_id')['play'].median()    
data.groupby('user_id')['artist'].count()    
- https://dacon.io/codeshare/582    
- https://ponyozzang.tistory.com/291 -> groupby  

Pandas 특정 조건을 만족하는 데이터 행 뽑기  
https://hogni.tistory.com/9


#### 딕셔너리 함수
get()       
-> dic.get(x)는 x라는 Key에 대응되는 Value를 돌려준다. 
-https://wikidocs.net/16



####  Matrix Factorization(MF)
MF는 CF(Collaborative Filtering)의 한 기법이다. MF는 user-item의 매트릭스에 비어있는 요소를 채우는 기술이다.     
추천 시스템의 다양한 모델 하나(행렬분해, MF)    
(m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어입니다.    
단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability(확장성)가 좋아서 많이 사용되는 모델      
Collaborative Filtering for Implicit Feedback Datasets 논문         
http://yifanhu.net/PUB/cf.pdf        
참고자료   
https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b   


#### CSR
-Compressed Sparse Row 의 약자   
-CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.    


#### SciPy(사이파이)
는 과학 컴퓨팅과 기술 컴퓨팅에 사용되는 자유-오픈 소스 파이썬 라이브러리.    
scipy.sparse.csr_matrix 
 사용법 https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
 
 
#### als(AlternatingLeastSquares) 모델
Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 방식이라고 한다.



# 질문

1.print(ratings['user_id'].nunique()) 인경우 6039값이 나옴    
movies_data.sort_values(by='user_id',ascending=True, inplace=False) 인경우 미지막 값은 6040 나옴   
->user_id은 첫번째값은 1, 마지막값은 6039값으로 설정됐다. 유니크한 값은 6040개가 되어야되는데 6039개다 ,그럼 중간에 한개의 값이 없다고 추측된다.(중간의 모든값을 보는방법이 있는가?)


2.my_moivelist = pd.DataFrame({'user_id': ['syj']*5, 'movie_id': my_movie_id, 'count':[5]*5})
에서 'user_id': [0]*5 으로 하면 굳이 인덱싱 과정을 하지 않아도 되지않나?
 -> csr에서 오류남 왜인지는 모르겠다.

3.join은 무엇인가?   
->두 개의 DataFrame을 합치는 것   
->Inner join : 두 DataFrame의 기준 컬럼에서 둘다 존재하는 데이터만 조인한다.
https://programmerpsy.tistory.com/17


4.MF를 바로 사용하지 않고 CSR 작업한 한 이유는?   
 -> MF같은 경우는 대부분 값이 0으로 채워진 행렬을 희소 행렬(Sparse Matrix)이다.
 희소 행렬은 너무 많은 불필요한 0 값으로 인해 메모리 낭비가 심하다. 또한 행렬의 크기가 커서 연산 시 시간도 많이 소모됩니다. 따라서 이런 희소 행렬을 메모리 낭비가 적도록 변환해야 하는데, 대표적인 방법이 COO 형식과 CSR 형식이다.
 https://bkshin.tistory.com/entry/NLP-7-%ED%9D%AC%EC%86%8C-%ED%96%89%EB%A0%AC-Sparse-Matrix-COO-%ED%98%95%EC%8B%9D-CSR-%ED%98%95%EC%8B%9D
 
5.협업필터링과 콘텐트기반 필터링 차이점   
 -> 협업 필터링은 다수의 사용자의 구매이력,시청이력 등 행동정보를 분석하여 사용자간 유사성 및 아이템 간 유사성을 파악하여 좋아할만한 항목을 추천해주는 기술이다.   
 -> 콘텐츠 기반 필터링은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악하여 추천해주는 기술    
https://blog.naver.com/navehag/222241684742

6.명시적(Explicit) 데이터와 암시적(Implicit) 데이터는 무엇인가?    
[Explicit Datasets]    
(1).유저가 자신의 선호도를 직접(Explicit) 표현   
(2).데이터를 얻기 힘들다   
[Implicit Datasets]   
(1)Implicit Data는 유저가 간접적(Implicit)으로 선호, 취향을 나타내는 데이터   
(2)부정적인 피드백이 없다(No Negative Feedback)    
(3)애초에 잡음이 많다(Inherently Noisy) :영상을 잠들고보던가,강제로볼수도 있기때문    
(4)수치는 신뢰도를 의미한다.(The numerical value of implicit feedback indicates confidence) /선호하는 영화같은경우 똑같은 영화를 자주본다.    
(5)Implicit-feedback Recommender System     
https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301